# Chapter 6: Metaclasses and Attributes

This chapter covers advanced Python features for customizing attribute access and class creation.

## Item 44: Use Plain Attributes Instead of Setter and Getter Methods

### Key Concept
Always start with simple public attributes. Only add @property when you need special behavior.

In [1]:
# ❌ NOT PYTHONIC - Explicit getters/setters
class OldResistor:
    def __init__(self, ohms):
        self._ohms = ohms

    def get_ohms(self):
        return self._ohms

    def set_ohms(self, ohms):
        self._ohms = ohms

r0 = OldResistor(50e3)
print('Before:', r0.get_ohms())
r0.set_ohms(10e3)
print('After:', r0.get_ohms())

Before: 50000.0
After: 10000.0


In [2]:
#  PYTHONIC - Simple public attributes
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0

r1 = Resistor(50e3)
r1.ohms = 10e3
r1.ohms += 5e3
print(f'Resistance: {r1.ohms} ohms')

Resistance: 15000.0 ohms


### Using @property for Special Behavior

In [3]:
# VoltageResistance Class - Commented Version

class VoltageResistance(Resistor):
    """
    A resistor subclass that manages voltage and automatically calculates
    current using Ohm's Law (I = V/R).

    This class demonstrates the use of @property decorators to create
    computed attributes with controlled access and side effects.
    """

    def __init__(self, ohms):
        """
        Initialize the VoltageResistance with a given resistance value.

        Args:
            ohms: The resistance value in ohms
        """
        # Call parent class constructor to set resistance value
        super().__init__(ohms)

        # Initialize private voltage attribute to 0
        # The underscore prefix indicates this is an internal attribute
        # that should be accessed via the voltage property
        self._voltage = 0

    @property
    def voltage(self):
        """
        Getter method for the voltage property.

        The @property decorator allows this method to be accessed like
        an attribute (r2.voltage) rather than a method call (r2.voltage()).

        Returns:
            The current voltage value in volts
        """
        return self._voltage

    @voltage.setter
    def voltage(self, voltage):
        """
        Setter method for the voltage property.

        The @voltage.setter decorator allows controlled assignment to the
        voltage attribute. When voltage is set, this method automatically
        recalculates the current based on Ohm's Law.

        Args:
            voltage: The new voltage value in volts

        Side Effects:
            - Updates the internal _voltage attribute
            - Automatically recalculates and updates self.current using I = V/R
        """
        # Store the new voltage value
        self._voltage = voltage

        # Automatically calculate and update current using Ohm's Law: I = V/R
        # This creates a dependency: whenever voltage changes, current updates
        self.current = self._voltage / self.ohms


# ============================================================================
# USAGE DEMONSTRATION
# ============================================================================

# Create a VoltageResistance instance with 1000 ohms (1 kilohm)
r2 = VoltageResistance(1e3)

# Display initial current (should be 0 since voltage defaults to 0)
# 0 volts / 1000 ohms = 0 amps
print(f'Before: {r2.current:.2f} amps')

# Set voltage to 10 volts
# This triggers the voltage.setter method, which:
#   1. Sets _voltage to 10
#   2. Calculates current: 10V / 1000Ω = 0.01A
r2.voltage = 10

# Display updated current (should be 0.01 amps)
# The current was automatically recalculated when voltage was set
print(f'After: {r2.current:.2f} amps')


# Key Concepts Demonstrated


Before: 0.00 amps
After: 0.01 amps



## 1. Property Decorator Pattern
- Transforms methods into managed attributes
- Provides controlled access to internal state
- Enables validation and side effects during assignment

## 2. Encapsulation
- Private attribute `_voltage` protects internal state
- Public interface through `voltage` property
- Enforces controlled modification

## 3. Automatic State Synchronization
- Setting voltage automatically updates current
- Maintains consistency between dependent attributes
- Implements reactive programming pattern

## 4. Inheritance
- Extends `Resistor` base class functionality
- Adds voltage management capability
- Preserves base class behavior through `super()`

## 5. Ohm's Law Implementation
- Mathematical relationship: I = V/R
- Current automatically calculated from voltage and resistance
- Demonstrates domain logic encapsulation

### Validation with @property

In [4]:
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError(f'ohms must be > 0; got {ohms}')
        self._ohms = ohms

r3 = BoundedResistance(1e3)
print(f'Valid: {r3.ohms}')

try:
    r3.ohms = 0
except ValueError as e:
    print(f'Error: {e}')

Valid: 1000.0
Error: ohms must be > 0; got 0


# NewBucket Implementation Analysis

## Token Bucket Rate Limiter with Detailed Commentary

This notebook provides a comprehensive analysis of the `NewBucket` class implementation, including:
- Fully commented source code
- Architectural design decisions
- Usage examples and demonstrations
- Potential improvements and best practices

In [5]:

## Part 1: Core Implementation with Comments

In [6]:
from datetime import datetime, timedelta

class NewBucket:
    """
    Token Bucket Rate Limiter Implementation

    This class implements a token bucket algorithm for rate limiting,
    where tokens (quota) are consumed over time and can be refilled
    periodically.
    """

    def __init__(self, period):
        """
        Initialize the bucket with a time period.

        Args:
            period: Time window in seconds for the bucket's rate limiting period
        """
        # Convert period to timedelta for proper time arithmetic
        self.period_delta = timedelta(seconds=period)

        # Track when the bucket was last reset/created
        self.reset_time = datetime.now()

        # Maximum capacity of the bucket (total tokens available)
        self.max_quota = 0

        # Tokens that have been used/consumed from the bucket
        self.quota_consumed = 0

    def __repr__(self):
        """
        String representation for debugging and logging.
        Shows current state of quota consumption.
        """
        return (f'NewBucket(max_quota={self.max_quota}, '
                f'quota_consumed={self.quota_consumed})')

    @property
    def quota(self):
        """
        Property getter: Calculate available quota.

        Returns:
            int: Available tokens (max_quota - consumed)

        Design Decision: Using a calculated property rather than storing
        available quota separately ensures consistency and eliminates
        potential synchronization issues.
        """
        return self.max_quota - self.quota_consumed

    @quota.setter
    def quota(self, amount):
        """
        Property setter: Handle quota assignment with three distinct scenarios.

        Args:
            amount: The new quota value to set

        This setter implements intelligent state transitions rather than
        direct assignment, maintaining internal consistency through
        calculated adjustments.
        """
        # Calculate the difference between current max and requested amount
        delta = self.max_quota - amount

        # SCENARIO 1: Reset/Empty the bucket
        if amount == 0:
            # Complete reset - both capacity and consumption go to zero
            self.quota_consumed = 0
            self.max_quota = 0

        # SCENARIO 2: Initial fill or capacity increase
        elif delta < 0:
            # Delta < 0 means amount > max_quota (requesting more than current capacity)
            # Assertion ensures we're starting fresh (no partial consumption)
            assert self.quota_consumed == 0, "Cannot increase capacity with consumed quota"
            # Set new maximum capacity
            self.max_quota = amount

        # SCENARIO 3: Consuming quota (normal usage)
        else:
            # Delta >= 0 means amount <= max_quota (consuming from available quota)
            # Assertion validates bucket integrity (consumed shouldn't exceed max)
            assert self.max_quota >= self.quota_consumed, "Quota consumed exceeds maximum"
            # Increase consumption by the delta amount
            self.quota_consumed += delta

---
## Part 2: Helper Function

In [7]:
def fill(bucket, amount):
    """
    Fill the bucket with tokens.

    This function adds tokens to the bucket by setting its quota.
    When filling an empty bucket, this triggers the capacity setting logic.

    Args:
        bucket: NewBucket instance to fill
        amount: Number of tokens to add
    """
    bucket.quota = amount

---
## Part 3: Basic Usage Demonstration

In [8]:
# Create a bucket with 60-second period
bucket = NewBucket(60)
print('Initial:', bucket)
print('Available quota:', bucket.quota)
print()

Initial: NewBucket(max_quota=0, quota_consumed=0)
Available quota: 0



In [9]:
# Fill the bucket with 100 tokens
fill(bucket, 100)
print('Filled:', bucket)
print('Available quota:', bucket.quota)
print()

Filled: NewBucket(max_quota=100, quota_consumed=0)
Available quota: 100



---
## Part 4: Demonstrating Token Consumption

In [10]:
# Consume some tokens by setting quota to a lower value
print(f"Before consumption: {bucket.quota} tokens available")

# Consume 30 tokens (setting quota to 70)
bucket.quota = 70
print(f"After consuming 30 tokens: {bucket}")
print(f"Available quota: {bucket.quota}")
print()

Before consumption: 100 tokens available
After consuming 30 tokens: NewBucket(max_quota=100, quota_consumed=30)
Available quota: 70



In [11]:
# Consume more tokens
bucket.quota = 20
print(f"After consuming 50 more tokens: {bucket}")
print(f"Available quota: {bucket.quota}")
print()

After consuming 50 more tokens: NewBucket(max_quota=100, quota_consumed=110)
Available quota: -10



---
## Part 5: Demonstrating Reset

In [12]:
# Reset the bucket
bucket.quota = 0
print(f"After reset: {bucket}")
print(f"Available quota: {bucket.quota}")
print()

After reset: NewBucket(max_quota=0, quota_consumed=0)
Available quota: 0



---
## Part 6: Design Analysis

### Key Design Decisions

#### 1. Calculated Property Pattern

The implementation uses a **calculated property** rather than storing available quota directly:

```python
@property
def quota(self):
    return self.max_quota - self.quota_consumed
```

**Advantages:**
- Single source of truth (derived value, not stored)
- Eliminates synchronization bugs between multiple state variables
- Always guarantees consistent state

**Alternative Approach (Not Used):**
```python
# Store available quota directly
self.available_quota = 100
```

**Why Avoided:**
- Would require maintaining three variables in sync
- Higher risk of inconsistent state
- More complex state management logic

#### 2. Tri-Modal State Transition Logic

The setter implements three distinct operational modes:

**Scenario 1: Reset (amount == 0)**
```python
if amount == 0:
    self.quota_consumed = 0
    self.max_quota = 0
```
- Complete bucket reset
- Use case: Clearing rate limits, initialization

**Scenario 2: Initial Fill (delta < 0)**
```python
elif delta < 0:
    assert self.quota_consumed == 0
    self.max_quota = amount
```
- Setting initial capacity or increasing capacity
- Assertion prevents capacity changes mid-consumption
- Use case: First fill, capacity upgrades

**Scenario 3: Consumption (delta >= 0)**
```python
else:
    assert self.max_quota >= self.quota_consumed
    self.quota_consumed += delta
```
- Normal token consumption
- Tracks cumulative token usage
- Use case: Rate limiting operations

#### 3. Assertion-Based Contract Enforcement

The implementation uses assertions to enforce preconditions:

**Assertion 1:**
```python
assert self.quota_consumed == 0
```
- Ensures bucket is empty before capacity changes
- Prevents invalid state transitions
- Catches programming errors during development

**Assertion 2:**
```python
assert self.max_quota >= self.quota_consumed
```
- Validates bucket integrity
- Catches overconsumption bugs
- Ensures logical consistency of internal state

**⚠️ Production Consideration:**
Assertions are disabled with `python -O` flag. For production systems, consider replacing with explicit validation and exception handling.

---
## Part 7: Real-World Use Case - API Rate Limiting

In [13]:
# Simulate API rate limiting: 100 requests per 60 seconds
api_bucket = NewBucket(60)
fill(api_bucket, 100)

print(f"API Rate Limiter initialized: {api_bucket.quota} requests available")
print()

# Simulate processing requests
def process_api_request(bucket, request_id):
    """Simulate processing an API request with rate limiting"""
    if bucket.quota > 0:
        # Consume one token
        bucket.quota = bucket.quota - 1
        print(f"✓ Request {request_id} processed. Remaining quota: {bucket.quota}")
        return True
    else:
        print(f"✗ Request {request_id} rejected. Rate limit exceeded!")
        return False

# Process some requests
for i in range(1, 6):
    process_api_request(api_bucket, i)

print(f"\nFinal state: {api_bucket}")

API Rate Limiter initialized: 100 requests available

✓ Request 1 processed. Remaining quota: 99
✓ Request 2 processed. Remaining quota: 97
✓ Request 3 processed. Remaining quota: 93
✓ Request 4 processed. Remaining quota: 85
✓ Request 5 processed. Remaining quota: 69

Final state: NewBucket(max_quota=100, quota_consumed=31)


---
## Part 8: Potential Improvements

### 1. Production-Ready Validation

Replace assertions with explicit exception handling:

In [14]:
class ProductionBucket(NewBucket):
    """Enhanced version with production-ready validation"""

    @property
    def quota(self):
        return self.max_quota - self.quota_consumed

    @quota.setter
    def quota(self, amount):
        """Setter with explicit exception handling instead of assertions"""
        delta = self.max_quota - amount

        if amount == 0:
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # Replace assertion with explicit validation
            if self.quota_consumed != 0:
                raise ValueError(
                    f"Cannot change capacity with consumed quota. "
                    f"Current consumption: {self.quota_consumed}"
                )
            self.max_quota = amount
        else:
            # Replace assertion with explicit validation
            if self.max_quota < self.quota_consumed:
                raise ValueError(
                    f"Invalid state: quota_consumed ({self.quota_consumed}) "
                    f"exceeds max_quota ({self.max_quota})"
                )
            self.quota_consumed += delta

# Test the production version
prod_bucket = ProductionBucket(60)
fill(prod_bucket, 50)
print(f"Production bucket: {prod_bucket}")

Production bucket: NewBucket(max_quota=50, quota_consumed=0)


### 2. Auto-Refill Functionality

Add automatic time-based refill when period expires:

In [16]:
class AutoRefillBucket(NewBucket):
    """Bucket with automatic time-based refill"""

    def _auto_refill(self):
        """Automatically refill tokens if period has elapsed"""
        now = datetime.now()
        elapsed = now - self.reset_time

        if elapsed >= self.period_delta:
            # Period has elapsed, reset consumption
            self.quota_consumed = 0
            self.reset_time = now
            print(f"[AUTO-REFILL] Bucket refilled to {self.max_quota} tokens")

    @property
    def quota(self):
        """Check for auto-refill before returning quota"""
        self._auto_refill()
        return self.max_quota - self.quota_consumed

    @quota.setter
    def quota(self, amount):
        """Redefine setter to maintain fill() compatibility"""
        delta = self.max_quota - amount
        if amount == 0:
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta

# Demonstrate auto-refill (requires time delay to see effect)
auto_bucket = AutoRefillBucket(60)
fill(auto_bucket, 100)
print(f"Auto-refill bucket created: {auto_bucket}")

Auto-refill bucket created: NewBucket(max_quota=100, quota_consumed=0)


### 3. Thread-Safe Implementation

Add locking for concurrent access:

In [17]:
from threading import Lock

class ThreadSafeBucket(NewBucket):
    """Thread-safe bucket implementation"""

    def __init__(self, period):
        super().__init__(period)
        self._lock = Lock()

    @property
    def quota(self):
        with self._lock:
            return self.max_quota - self.quota_consumed

    @quota.setter
    def quota(self, amount):
        with self._lock:
            delta = self.max_quota - amount

            if amount == 0:
                self.quota_consumed = 0
                self.max_quota = 0
            elif delta < 0:
                assert self.quota_consumed == 0
                self.max_quota = amount
            else:
                assert self.max_quota >= self.quota_consumed
                self.quota_consumed += delta

# Create thread-safe bucket
safe_bucket = ThreadSafeBucket(60)
fill(safe_bucket, 100)
print(f"Thread-safe bucket: {safe_bucket}")

Thread-safe bucket: NewBucket(max_quota=100, quota_consumed=0)


---
## Part 9: Summary

### Key Takeaways

**Strengths of the Implementation:**
1. **Calculated Property Pattern** - Eliminates state synchronization issues
2. **Explicit State Transitions** - Clear tri-modal logic for different operations
3. **Defensive Programming** - Assertions catch logic errors during development
4. **Clean Interface** - Property decorator provides intuitive API

**Areas for Enhancement:**
1. **Missing Refill Logic** - No automatic token restoration over time
2. **Assertion Fragility** - Validation disappears in optimized deployments
3. **No Concurrency Control** - Not safe for multi-threaded environments
4. **Incomplete API** - Missing convenience methods for consumption

**Recommended Use Cases:**
- API rate limiting
- Bandwidth throttling
- Request quotas
- Resource allocation control
- Traffic shaping algorithms

---

## Item 45: Consider @property Instead of Refactoring Attributes

### Migrating to Calculated Properties

In [18]:
from datetime import datetime, timedelta

class Bucket:
    def __init__(self, period):
        # Store the time period as a timedelta object for easy comparison
        self.period_delta = timedelta(seconds=period)
        # Initialize the reset time to current moment
        self.reset_time = datetime.now()
        # Initialize quota counter to zero
        self.quota = 0

    def __repr__(self):
        # Provide readable string representation of bucket state
        return f'Bucket(quota={self.quota})'

def fill(bucket, amount):
    # Get current timestamp for period comparison
    now = datetime.now()
    # Check if current period has expired by comparing time elapsed
    if (now - bucket.reset_time) > bucket.period_delta:
        # Reset quota to zero when new period begins
        bucket.quota = 0
        # Update reset_time to mark start of new period
        bucket.reset_time = now
    # Add the specified amount to current quota
    bucket.quota += amount

def deduct(bucket, amount):
    # Get current timestamp for period validation
    now = datetime.now()
    # Reject deduction if current period has expired (quota reset needed)
    if (now - bucket.reset_time) > bucket.period_delta:
        return False
    # Reject deduction if insufficient quota available
    if bucket.quota - amount < 0:
        return False
    # Deduct the amount from available quota
    bucket.quota -= amount
    # Return success indicator
    return True

# Create a bucket with 60-second period window
bucket = Bucket(60)
# Add 100 units to the bucket's quota
fill(bucket, 100)
# Display current bucket state
print(bucket)

# Attempt to deduct 99 units from quota
if deduct(bucket, 99):
    # Confirm successful deduction
    print('Had 99 quota')
# Display bucket state after deduction
print(bucket)

Bucket(quota=100)
Had 99 quota
Bucket(quota=1)


### Improved Version with @property

In [19]:
class NewBucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0

    def __repr__(self):
        return (f'NewBucket(max_quota={self.max_quota}, '
                f'quota_consumed={self.quota_consumed})')

    @property
    def quota(self):
        return self.max_quota - self.quota_consumed

    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta

bucket = NewBucket(60)
print('Initial', bucket)
fill(bucket, 100)
print('Filled', bucket)

Initial NewBucket(max_quota=0, quota_consumed=0)
Filled NewBucket(max_quota=100, quota_consumed=0)


---

## Item 46: Use Descriptors for Reusable @property Methods

### The Problem: @property Can't Be Reused

In [20]:
# Problem: Repetitive @property code
class Homework:
    def __init__(self):
        self._grade = 0

    @property
    def grade(self):
        return self._grade

    @grade.setter
    def grade(self, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._grade = value

galileo = Homework()
galileo.grade = 95
print(f'Grade: {galileo.grade}')

Grade: 95


### Solution: Descriptors

In [21]:
from weakref import WeakKeyDictionary

class Grade:
    def __init__(self):
        self._values = WeakKeyDictionary()

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._values.get(instance, 0)

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value

class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 75

print(f'First: {first_exam.writing_grade}')
print(f'Second: {second_exam.writing_grade}')

First: 82
Second: 75


---

## Item 47: Use __getattr__, __getattribute__, and __setattr__ for Lazy Attributes

### __getattr__ for Lazy Loading

In [22]:
class LazyRecord:
    def __init__(self):
        self.exists = 5

    def __getattr__(self, name):
        value = f'Value for {name}'
        setattr(self, name, value)
        return value

data = LazyRecord()
print('Before:', data.__dict__)
print('foo:', data.foo)
print('After:', data.__dict__)

Before: {'exists': 5}
foo: Value for foo
After: {'exists': 5, 'foo': 'Value for foo'}


### __getattribute__ for All Access

In [23]:
class ValidatingRecord:
    def __init__(self):
        self.exists = 5

    def __getattribute__(self, name):
        print(f'* Called __getattribute__({name!r})')
        try:
            value = super().__getattribute__(name)
            print(f'* Found {name!r}, returning {value!r}')
            return value
        except AttributeError:
            value = f'Value for {name}'
            print(f'* Setting {name!r} to {value!r}')
            setattr(self, name, value)
            return value

data = ValidatingRecord()
print('exists:', data.exists)
print('First foo:', data.foo)
print('Second foo:', data.foo)

* Called __getattribute__('exists')
* Found 'exists', returning 5
exists: 5
* Called __getattribute__('foo')
* Setting 'foo' to 'Value for foo'
First foo: Value for foo
* Called __getattribute__('foo')
* Found 'foo', returning 'Value for foo'
Second foo: Value for foo


### __setattr__ for Write Interception

In [24]:
class LoggingSavingRecord:
    def __setattr__(self, name, value):
        print(f'* Called __setattr__({name!r}, {value!r})')
        super().__setattr__(name, value)

data = LoggingSavingRecord()
print('Before:', data.__dict__)
data.foo = 5
print('After:', data.__dict__)
data.foo = 7
print('Finally:', data.__dict__)

Before: {}
* Called __setattr__('foo', 5)
After: {'foo': 5}
* Called __setattr__('foo', 7)
Finally: {'foo': 7}


---

## Item 48: Validate Subclasses with __init_subclass__

### Metaclass Approach (Old Way)

In [25]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print(f'* Running {meta}.__new__ for {name}')
        print('Bases:', bases)
        return type.__new__(meta, name, bases, class_dict)

class MyClass(metaclass=Meta):
    stuff = 123
    def foo(self):
        pass

class MySubclass(MyClass):
    other = 567
    def bar(self):
        pass

* Running <class '__main__.Meta'>.__new__ for MyClass
Bases: ()
* Running <class '__main__.Meta'>.__new__ for MySubclass
Bases: (<class '__main__.MyClass'>,)


### __init_subclass__ Approach (Better)

In [26]:
class BetterPolygon:
    sides = None

    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.sides < 3:
            raise ValueError('Polygons need 3+ sides')

    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180

class Hexagon(BetterPolygon):
    sides = 6

print(f'Hexagon angles: {Hexagon.interior_angles()}')

try:
    class Point(BetterPolygon):
        sides = 1
except ValueError as e:
    print(f'Error: {e}')

Hexagon angles: 720
Error: Polygons need 3+ sides


### Multiple Inheritance with __init_subclass__

In [27]:
class Filled:
    color = None

    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.color not in ('red', 'green', 'blue'):
            raise ValueError('Fills need a valid color')

class RedTriangle(Filled, BetterPolygon):
    color = 'red'
    sides = 3

ruddy = RedTriangle()
print(f'Created: {ruddy.__class__.__name__}')

Created: RedTriangle


---

## Item 49: Register Class Existence with __init_subclass__

### Automatic Class Registration

In [28]:
import json

class Serializable:
    def __init__(self, *args):
        self.args = args

    def serialize(self):
        return json.dumps({'args': self.args})

class Point2D(Serializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Point2D({self.x}, {self.y})'

point = Point2D(5, 3)
print('Object:', point)
print('Serialized:', point.serialize())

Object: Point2D(5, 3)
Serialized: {"args": [5, 3]}


### With __init_subclass__ Registration

In [29]:
registry = {}

class BetterSerializable:
    def __init__(self, *args):
        self.args = args

    def __init_subclass__(cls):
        super().__init_subclass__()
        registry[cls.__name__] = cls

    def serialize(self):
        return json.dumps({
            'class': self.__class__.__name__,
            'args': self.args
        })

    def __repr__(self):
        name = self.__class__.__name__
        args_str = ', '.join(str(x) for x in self.args)
        return f'{name}({args_str})'

def deserialize(data):
    params = json.loads(data)
    name = params['class']
    target_class = registry[name]
    return target_class(*params['args'])

class Vector1D(BetterSerializable):
    def __init__(self, magnitude):
        super().__init__(magnitude)
        self.magnitude = magnitude

before = Vector1D(6)
data = before.serialize()
after = deserialize(data)
print(f'Before: {before}')
print(f'After: {after}')

Before: Vector1D(6)
After: Vector1D(6)


---

## Item 50: Annotate Class Attributes with __set_name__

### Descriptor with __set_name__

In [30]:
class Field:
    def __init__(self):
        self.name = None
        self.internal_name = None

    def __set_name__(self, owner, name):
        self.name = name
        self.internal_name = '_' + name

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return getattr(instance, self.internal_name, '')

    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)

class Customer:
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()

cust = Customer()
print('Before:', cust.__dict__)
cust.first_name = 'Euclid'
print('After:', cust.__dict__)

Before: {}
After: {'_first_name': 'Euclid'}


---

## Item 51: Prefer Class Decorators Over Metaclasses

### Class Decorator for Tracing

In [32]:
from functools import wraps

def trace_func(func):
    if hasattr(func, 'tracing'):
        return func

    @wraps(func)
    def wrapper(*args, **kwargs):
        result = None
        try:
            result = func(*args, **kwargs)
            return result
        except Exception as e:
            result = e
            raise
        finally:
            print(f'{func.__name__}({args!r}, {kwargs!r}) -> {result!r}')

    wrapper.tracing = True
    return wrapper

def trace(klass):
    for key in dir(klass):
        value = getattr(klass, key)
        # Skip special methods and check if it's a callable method (not already wrapped)
        if callable(value) and not key.startswith('__'):
            wrapped = trace_func(value)
            # Use staticmethod to wrap the function before setting it back
            setattr(klass, key, staticmethod(wrapped).__func__)
    return klass

@trace
class TraceDict(dict):
    pass

trace_dict = TraceDict([('hi', 1)])
trace_dict['there'] = 2
trace_dict['hi']

1

---

## Chapter Summary

### Key Concepts

| Item | Concept | Use Case |
|------|---------|----------|
| 44 | Plain attributes + @property | Start simple, add behavior later |
| 45 | @property for refactoring | Migrate without breaking API |
| 46 | Descriptors | Reusable @property logic |
| 47 | __getattr__, __setattr__ | Lazy loading, proxies |
| 48 | __init_subclass__ | Subclass validation |
| 49 | Class registration | Automatic type registry |
| 50 | __set_name__ | Descriptor introspection |
| 51 | Class decorators | Composable class extensions |